In [18]:
%load_ext autoreload
%autoreload 2

import os, re, json, ast
from llm_api_calls import send_message_to_gemini_async
from km_utils import get_ToCs, split_dict_by_size, get_content_from_articles_response, split_list_by_size
from coap_prompts import *
from parallel_descriptions import RateLimiter
from pprint import pprint
import asyncio
from test_cases import test_cases

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
# pip install spacy rank_bm25
# python -m spacy download ru_core_news_sm

In [20]:
import spacy
from rank_bm25 import BM25Okapi

# Load the Russian language model
spacy_nlp = spacy.load("ru_core_news_sm")

# Function to preprocess documents and queries
def spacy_preprocess(text):
    doc = spacy_nlp(text)
    lemmatized = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return lemmatized

ToCs = get_ToCs(base_path = 'coap_map')

all_articles = {}
ToC_articles = ToCs['ToC_articles']
for section in ToC_articles.keys():
    for chapter in ToC_articles[section].keys():
        for article in ToC_articles[section][chapter].keys():
            all_articles[ToC_articles[section][chapter][article]['article_title']] = ToC_articles[section][chapter][article]['article_content']

processed_articles = {article_id: spacy_preprocess(doc) for article_id, doc in all_articles.items()}

In [21]:
bm25 = BM25Okapi(processed_articles.values())

In [36]:
from parallel_descriptions import RateLimiter

async def retrieve_articles(user_query_ori, top_n=10, rate_limiter=None):
    response = await get_perefrased_query_bm_25(USER_QUERY=user_query_ori, rate_limiter=rate_limiter)
    try: 
        json_response = ast.literal_eval(response['text_response'])
    except: 
        json_response = {'Термины': [], 'Вариации вопросов': []}

    termin = " ".join(json_response['Термины']) #+ " ".join([x for x in json_response['Вариации вопросов']])
    # termin = " ".join([x for x in json_response['Вариации вопросов']])
    tokenized_termin = spacy_preprocess(termin)
    scores = bm25.get_scores(tokenized_termin)

    # Создание списка пар (ключ, оценка)
    doc_scores = [(key, scores[i]) for i, (key, doc) in enumerate(all_articles.items())]

    # Сортировка списка по оценкам в убывающем порядке
    top_n_doc_scores = sorted(doc_scores, key=lambda x: x[1], reverse=True)[:top_n]

    # Получение идентификаторов статей из top_n пар
    article_ids = []
    for doc_key, _ in top_n_doc_scores:
        pattern = re.compile(r'Статья\s+\d+(\.\d+)*(-\d+)?\.')
        match = pattern.search(doc_key.strip())
        article_id = 'не найдено'
        if match:
            article_id = match.group().strip()
        article_ids.append(article_id)

    return {'article_ids':article_ids, 'json_response':json_response}


from test_cases import test_cases

async def process_test_case(test_case, rate_limiter):
    user_query_ori = test_case['user_query_ori']
    needed_articles = set(test_case['needed_articles'])
    retrieved_articles = await retrieve_articles(user_query_ori, top_n=len(needed_articles), rate_limiter=rate_limiter)
    
    retrieved_article_ids = retrieved_articles['article_ids']
    test_case['retrieved_articles'] = []

    for position, retrieved_article in enumerate(retrieved_article_ids, start=1):
        test_case['json_response'] = retrieved_articles['json_response']
        for needed_article in needed_articles:
            if needed_article == retrieved_article:
                test_case['retrieved_articles'].append({'article': needed_article, 'position': position})
                break  # Assuming each needed article is unique

rate_limiter = RateLimiter(20, 25)

tasks = []
for ind, test_case in enumerate(test_cases):
    tasks.append(process_test_case)

tasks = [process_test_case(test_case, rate_limiter) for test_case in test_cases]
await asyncio.gather(*tasks)
test_cases


Error 429 Resource has been exhausted (e.g. check quota)., retrying in 1 seconds... (Attempt 1/10)
Error 429 Resource has been exhausted (e.g. check quota)., retrying in 1 seconds... (Attempt 1/10)
Error 429 Resource has been exhausted (e.g. check quota)., retrying in 1 seconds... (Attempt 1/10)
Error 429 Resource has been exhausted (e.g. check quota)., retrying in 2 seconds... (Attempt 2/10)
Error 429 Resource has been exhausted (e.g. check quota)., retrying in 2 seconds... (Attempt 2/10)
Ты знающий Юрист в Российской Федерации который помогает сформулировать вопрос на яыке используемом в КОАП РФ и других законодательных документах, таких как конституция и УК РФ.
Выводи ответ в формате json.
1. Ключ "Вариации вопросов". Инструкция: Предложи 4 варианта разнообразных и лаконичных формулировок вопроса, не повторяйся.
Учти что есть многозначные слова, "повестка" может означать как повестка в суд, так и в военкомат. Для многозначных слов приводи их юридические синонимы.
Обязательно сделай 

[{'user_query_ori': 'что будет за дискредитацию армии рф?',
  'needed_articles': ['Статья 20.3.3.'],
  'retrieved_articles': [{'article': 'Статья 20.3.3.', 'position': 1}],
  'json_response': {'Вариации вопросов': ['Какова ответственность за дискредитацию вооружённых сил РФ?',
    'Какие возможны формы наказания и штрафные санкции по статье о дискредитации вооружённых сил РФ?',
    'В чем особенность состава правонарушения по ст. 20.3.3 КоАП РФ, предусматривающей ответственность за дискредитацию вооружённых сил РФ?',
    'Какие законодательные основания были использованы при создании статьи о дискредитации вооружённых сил РФ?'],
   'Термины': ['дискредитация',
    'публичное',
    'предусмотренный уголовным кодексом',
    'правонарушение',
    'вооруженные силы',
    'штраф',
    'форма наказания']}},
 {'user_query_ori': 'штраф за пересечение стоп-линии красный сигнал светофора',
  'needed_articles': ['Статья 12.12.'],
  'retrieved_articles': [{'article': 'Статья 12.12.', 'position': 1

In [37]:
def calculate_metrics(test_cases, k=10):
    mrr_sum = 0
    total_recall_at_k = 0
    for test_case in test_cases:
        needed_articles = set(test_case['needed_articles'])
        retrieved_articles_positions = {ra['article']: ra['position'] for ra in test_case.get('retrieved_articles', [])}
        
        # MRR calculation
        ranks = [1/position for article, position in retrieved_articles_positions.items() if article in needed_articles]
        if ranks:
            mrr_sum += max(ranks)  # Only the first occurrence of each needed article contributes to MRR
        
        # Recall at K calculation
        hits_at_k = sum(1 for article, position in retrieved_articles_positions.items() if article in needed_articles and position <= k)
        total_recall_at_k += hits_at_k / len(needed_articles) if needed_articles else 0

    mrr = mrr_sum / len(test_cases)
    recall_at_k = total_recall_at_k / len(test_cases)
    return {'mrr':mrr, 'recall_at_k':recall_at_k}
calculate_metrics(test_cases, k=10)


{'mrr': 0.6388888888888888, 'recall_at_k': 0.6111111111111112}